# About
This script is used to convert MobileNet EdgeTPU to Core ML

# Download model and setup enviroments

Download the TF model from this link:  
https://storage.cloud.google.com/mobilenet_edgetpu/checkpoints/mobilenet_edgetpu_224_1.0.tgz

In [3]:
!pip install coremltools==4.1

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=8b992751f96637404a8e95516a6a843b890ac098e046d4c326d4693c21950607
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [4]:
%tensorflow_version 2.x

In [5]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
2.5.0


4.1


# Convert from TF to Core ML

In [6]:
from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile

from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants

def load_frozen_graph(path):
  with tf.io.gfile.GFile(path, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    return graph_def

def optimize_graph(graph_def, 
                   input_node_name, 
                   output_node_name,
                   export_dir):
  gdef = strip_unused_lib.strip_unused(
          input_graph_def = graph_def,
          input_node_names = [input_node_name],
          output_node_names = [output_node_name],
          placeholder_type_enum = dtypes.float32.as_datatype_enum)

  builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_dir)
  sigs = {}
  
  with tf.compat.v1.Session(graph=tf.Graph()) as sess:
      tf.import_graph_def(gdef, name="")
      g = tf.compat.v1.get_default_graph()
      input_name = g.get_tensor_by_name(input_node_name+':0')
      output_name = g.get_tensor_by_name(output_node_name+':0')

      sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
        tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(
          {"input": input_name}, {"output": output_name})

      builder.add_meta_graph_and_variables(sess, [tag_constants.SERVING], signature_def_map=sigs)
      builder.save()

In [7]:
def convert_to_coreml(saved_model_dir,
                      input_name,
                      output_name,
                      coreml_file):
  # Download class labels (from a separate file)
  import urllib
  label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
  class_labels = urllib.request.urlopen(label_url).read().splitlines()
  assert len(class_labels) == 1001

  # make sure entries of class_labels are strings
  for i, label in enumerate(class_labels):
    if isinstance(label, bytes):
      class_labels[i] = label.decode("utf8")

  image_input = ct.ImageType(shape=(1, 224, 224, 3),
                             bias=[-1,-1,-1],
                             scale=1/127)

  # set class labels
  classifier_config = ct.ClassifierConfig(class_labels)

  # Convert the model using the Unified Conversion API
  model = ct.convert(
      saved_model_dir,
      source='tensorflow', 
      inputs=[image_input], 
      classifier_config=classifier_config,
  )

  print(model.input_description, '->', model.output_description)

  # Set feature descriptions (these show up as comments in XCode)
  model.input_description["images"] = "Input image to be classified"
  model.output_description["Softmax"] = "Most likely image category"

  # Set model author name
  model.author = 'Converted from TF to Core ML by Anh'

  # Set a short description for the Xcode UI
  model.short_description = '''
  MobilenetEdgeTPU from 
  https://storage.cloud.google.com/mobilenet_edgetpu/checkpoints/mobilenet_edgetpu_224_1.0.tgz
  '''

  # Set a version for the model
  model.version = "2021-07-30"

  spec = model._spec

  ct.utils.rename_feature(spec, input_name, "image")
  ct.utils.rename_feature(spec, output_name, "classLabelProbs")

  print(model.input_description, '->', model.output_description)

  model.save(coreml_file)

In [8]:
# Download the model from 
# https://storage.cloud.google.com/mobilenet_edgetpu/checkpoints/mobilenet_edgetpu_224_1.0.tgz

input_name = 'images'
output_name = 'Softmax'
export_dir = 'optimized_saved_model'
graph_def = load_frozen_graph('/content/drive/MyDrive/models/mobilenet_edgetpu_224_1.0/frozen_graph.pb')
optimize_graph(graph_def,
               input_name,
               output_name,
               export_dir)

convert_to_coreml(export_dir,
                  input_name,
                  output_name,
                  coreml_file='MobilenetEdgeTPU_Anh.mlmodel')

Translating MIL ==> MLModel Ops: 100%|██████████| 826/826 [00:00<00:00, 1183.09 ops/s]


Features(images) -> Features(Softmax,classLabel)
Features(image) -> Features(classLabelProbs,classLabel)
